## Write a Web Service

Wrap the output of the second exercise in a web service that returns the data in JSON format (instead of printing to the standard output).

The web service should accept a parameter n>0. For the top 10 airports, n is 10. For the X top airports, n is X 


In [1]:
%%time
import pandas as pd
path='/home/dsc/Data/challenge'
#file_name='bookings.sample.csv.bz2'
file_name='bookings.csv.bz2'
filepath=path+'/'+file_name

bc = pd.read_csv(filepath,sep='^',usecols=['arr_port','pax'], chunksize=1000000)


sum_chunks=pd.DataFrame()
for i,chunk in enumerate(bc):
    print i
    chunk_airports=chunk.dropna().groupby(['arr_port']).sum().reset_index()
    sum_chunks=sum_chunks.append(chunk_airports)

topairports=sum_chunks.groupby('arr_port').pax.sum().sort_values(ascending=False).reset_index()

#.head(10) 
#topairports

0
1
2
3
4
5
6
7
8
9
10
CPU times: user 3min 33s, sys: 1.21 s, total: 3min 34s
Wall time: 3min 35s


In [2]:
topairports.arr_port.head()

0    LHR     
1    MCO     
2    LAX     
3    LAS     
4    JFK     
Name: arr_port, dtype: object

In [3]:
topairports2=topairports.copy()

In [5]:
list(topairports2.arr_port.head())

['LHR     ', 'MCO     ', 'LAX     ', 'LAS     ', 'JFK     ']

map= works element wise on series

apply= works on row/columns of DF (def=columns; axis=1 => rows); works on DF of group by

applymap = works element wise on DF

In [13]:
topairports2.arr_port=topairports2.arr_port.map(str.strip)

In [14]:
list(topairports2.arr_port.head())

['LHR', 'MCO', 'LAX', 'LAS', 'JFK']

In [22]:
#topairports.pax=pd.to_numeric(topairports.pax)

In [28]:
topairports2.pax=topairports2.pax.map(lambda x:int(x))

In [48]:
topairports2.to_csv('top_airports.csv',sep='^', index=False)

## Web Service

In [1]:
from flask import Flask

In [2]:
app = Flask("My first web service")
@app.route('/hello', methods=['GET'])
def get_hello():
    return "Hello DS from the service!"

In [3]:
app.run()

In [4]:
del app

In [6]:
import pandas as pd
top=pd.read_csv('top_airports.csv',sep='^')
top.head()

,arr_port,pax
0,LHR,88809
1,MCO,70930
2,LAX,70530
3,LAS,69630
4,JFK,66270


In [50]:
top.head(10).to_json()

'{"arr_port":{"0":"LHR","1":"MCO","2":"LAX","3":"LAS","4":"JFK","5":"CDG","6":"BKK","7":"MIA","8":"SFO","9":"DXB"},"pax":{"0":88809,"1":70930,"2":70530,"3":69630,"4":66270,"5":64490,"6":59460,"7":58150,"8":58000,"9":55590}}'

In [7]:
app2 = Flask("Top Airports")
@app2.route('/top/<int:n>',methods=['GET'])

def get_top(n):
    d = top.head(n)
    return d.to_json()

In [11]:
app2.run()

In [9]:
del app2

In [10]:
app2 = Flask("Top Airports")

@app2.route('/top<int:n>',methods=['GET'])
def get_top(n):
    d = top.head(n)
    return d.to_json()

@app2.route('/', methods=['GET'])
def get_hello():
    return "Hello from my second web service!"

In [12]:
app2.run()